In [99]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F


ModuleNotFoundError: No module named 'sqlContext'

In [138]:

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [5]:
df.write.parquet('zones')


In [16]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv


In [9]:
!wc -l fhvhv_tripdata_2021-01.csv


11908469 fhvhv_tripdata_2021-01.csv


In [11]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [12]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [13]:
df.schema


StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [14]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [15]:
!head -n 10 head.csv

In [17]:
df_pandas = pd.read_csv("head.csv")

In [18]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [20]:
spark.createDataFrame(df_pandas).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [21]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [6]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [35]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])
df_zone = spark.read \
    .option("header", "true") \
    .schema(schema)\
    .csv('fhvhv_tripdata_2021-01.csv')

In [36]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [37]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [38]:
df=df.repartition(24)

In [39]:
df.write.parquet("fhvhv/2021/01/")

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df1 = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

df1.show()

22/02/26 10:35:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 10:35:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [8]:
df1 = spark.read \
    .option("header", "true") \
    .schema(schema)\
    .csv('fhvhv_tripdata_2021-02.csv')

df1.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [9]:
df1.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,StringType,true)))

In [10]:
df1=df1.repartition(24)

In [11]:
df1.write.parquet("fhvhv/2021/02/")

In [ ]:
df1.select('pickup_datetime').filter(df1.pickup_datetime)

In [17]:
df2 = spark.read.parquet('fhvhv/2021/02/')

In [283]:
df2.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [25]:
df2.select('pickup_datetime')

DataFrame[pickup_datetime: timestamp]

In [28]:
df2.select('hvfhs_license_num','pickup_datetime').filter(df2.pickup_datetime=="2021-02-06 01:40:34").show()

+-----------------+-------------------+
|hvfhs_license_num|    pickup_datetime|
+-----------------+-------------------+
|           HV0005|2021-02-06 01:40:34|
|           HV0003|2021-02-06 01:40:34|
|           HV0003|2021-02-06 01:40:34|
+-----------------+-------------------+



22/02/26 12:03:58 WARN TaskSetManager: Lost task 4.0 in stage 12.0 (TID 49, de-zoom-greg.europe-west1-b.c.mimetic-core-338720.internal, executor driver): TaskKilled (Stage cancelled)


In [47]:
df2\
    .select('hvfhs_license_num','pickup_datetime')\
    .filter((df2.pickup_datetime>="2021-02-15") & (df2.pickup_datetime<"2021-02-16") )\
    .show()

+-----------------+-------------------+
|hvfhs_license_num|    pickup_datetime|
+-----------------+-------------------+
|           HV0005|2021-02-15 22:54:33|
|           HV0003|2021-02-15 00:21:01|
|           HV0005|2021-02-15 21:25:07|
|           HV0003|2021-02-15 10:36:49|
|           HV0005|2021-02-15 21:58:37|
|           HV0003|2021-02-15 14:50:53|
|           HV0003|2021-02-15 18:43:20|
|           HV0003|2021-02-15 23:15:54|
|           HV0004|2021-02-15 22:19:27|
|           HV0003|2021-02-15 15:50:01|
|           HV0004|2021-02-15 12:57:15|
|           HV0005|2021-02-15 00:13:21|
|           HV0005|2021-02-15 21:31:16|
|           HV0003|2021-02-15 10:43:59|
|           HV0003|2021-02-15 00:40:14|
|           HV0003|2021-02-15 08:37:13|
|           HV0003|2021-02-15 09:10:22|
|           HV0005|2021-02-15 11:50:17|
|           HV0003|2021-02-15 13:15:27|
|           HV0003|2021-02-15 16:25:35|
+-----------------+-------------------+
only showing top 20 rows



In [44]:
df2\
    .select('hvfhs_license_num','pickup_datetime')\
    .filter((df2.pickup_datetime>="2021-02-15") & (df2.pickup_datetime<"2021-02-16") )\
    .count()

367170

In [53]:
timeFmt = "yyyy-MM-dd' 'HH:mm:ss"
timeDiff = (F.unix_timestamp('dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('pickup_datetime', format=timeFmt))

In [111]:
timeDiff =F.to_timestamp(df2.dropoff_datetime) - F.to_timestamp(df2.pickup_datetime)

In [ ]:
F.to_timestamp

In [ ]:
unix_timestamp(df("stringCol"), pattern).cast("timestamp")

In [80]:
timeDiff= df2.dropoff_datetime- df2.pickup_datetime

In [109]:
def timeDiff(drop, pickup):
#     if (pickup or drop)==0:
#         return 0
#     else:
        return drop-pickup

timeDiffudf= F.udf(timeDiff, returnType=types.TimestampType())

In [ ]:
# df2\
#     .select('pickup_datetime', 'dropoff_datetime')\
#     .withColumn('trip_length', timeDiffudf(df2.dropoff_datetime, df2.pickup_datetime))\
#     .sort('pickup_datetime')\
#     .show()


In [112]:
timeDiff =F.to_timestamp(df2.dropoff_datetime) - F.to_timestamp(df2.pickup_datetime)

In [115]:
timeDiff= F.unix_timestamp(df2.dropoff_datetime- df2.pickup_datetime)

In [118]:
timeFmt = "yyyy-MM-dd' 'HH:mm:ss"
timeDiff = (F.unix_timestamp('dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('pickup_datetime', format=timeFmt))

In [ ]:
F.desc

In [125]:
timeFmt = "yyyy-MM-dd' 'HH:mm:ss"
timeDiff = (F.unix_timestamp('dropoff_datetime', format=timeFmt)
            - F.unix_timestamp('pickup_datetime', format=timeFmt))
df2\
    .select('pickup_datetime', 'dropoff_datetime')\
    .withColumn('trip_length', timeDiff)\
    .sort(F.desc('trip_length'))\
    .show()


+-------------------+-------------------+-----------+
|    pickup_datetime|   dropoff_datetime|trip_length|
+-------------------+-------------------+-----------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|      75540|
|2021-02-17 15:54:53|2021-02-18 07:48:34|      57221|
|2021-02-20 12:08:15|2021-02-21 00:22:14|      44039|
|2021-02-03 20:24:25|2021-02-04 07:41:58|      40653|
|2021-02-19 23:17:44|2021-02-20 09:44:01|      37577|
|2021-02-25 17:13:35|2021-02-26 02:57:05|      35010|
|2021-02-20 01:36:13|2021-02-20 11:16:19|      34806|
|2021-02-18 15:24:19|2021-02-19 01:01:11|      34612|
|2021-02-18 01:31:20|2021-02-18 11:07:15|      34555|
|2021-02-10 20:51:39|2021-02-11 06:21:08|      34169|
|2021-02-10 01:56:17|2021-02-10 10:57:33|      32476|
|2021-02-25 09:18:18|2021-02-25 18:18:57|      32439|
|2021-02-21 19:59:13|2021-02-22 04:56:16|      32223|
|2021-02-09 18:36:13|2021-02-10 03:31:00|      32087|
|2021-02-06 09:48:09|2021-02-06 18:32:16|      31447|
|2021-02-02 09:42:30|2021-02

In [132]:
df2\
    .select('hvfhs_license_num')\
    .groupBy('hvfhs_license_num')\
    .withColumn('count', F.count('hvfhs_license_num'))\
    .show()

AttributeError: 'GroupedData' object has no attribute 'withColumn'

In [309]:
df2\
    .select('hvfhs_license_num')\
    .groupBy('hvfhs_license_num').agg(F.count("*"))\
    .show()

+-----------------+--------+
|hvfhs_license_num|count(1)|
+-----------------+--------+
|           HV0004|   86018|
|           HV0005| 3237166|
|           HV0003| 8290758|
+-----------------+--------+



In [137]:
df2\
    .select('*')\
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [178]:
cond = [df2.PULocationID == df.LocationID, df2.DOLocationID == df.LocationID]
cond1 = [df2.PULocationID == df.LocationID]
cond2= [df2.DOLocationID == df.LocationID]
df2\
    .join(df, cond1, "left")\
    .select("hvfhs_license_num", "pickup_datetime", "PULocationID", "DOLocationID", "LocationID", "Borough")\
    .join(df, cond2, "left")\
    .select("hvfhs_license_num", "pickup_datetime", "PULocationID", "DOLocationID", "LocationID", "Borough")\
    .show()

AnalysisException: Reference 'LocationID' is ambiguous, could be: LocationID, LocationID.;

In [189]:
cond = [df2.PULocationID == df.LocationID, df2.DOLocationID == df.LocationID]
cond1 = [df2.PULocationID == df.LocationID]
cond2= [df2.DOLocationID == df.LocationID]
df2\
    .join(df, cond1, "left").alias("PU")\
    .join(df, cond2, "left")\
    .select("PULocationID", PU.LocationID)\
    .show()

NameError: name 'PU' is not defined

In [206]:
#.withColumn("PU_loc").
df_PU = df.alias("df_PU")
df_DO = df.alias("df_DO")
cond1 = [df2.PULocationID == df_PU.LocationID]
cond2= [df2.DOLocationID == df_DO.LocationID]

df2\
    .join(df_PU , cond1, "inner")\
    .join(df_DO, cond2, "inner")\
    .show()

AnalysisException: Column LocationID#1042 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.;

In [201]:
cond1 = [df2.PULocationID == df.LocationID]
cond2= [df2.DOLocationID == df.LocationID]
df2\
    .join(df.alias("PU_Borough"), cond1, "inner")\
    .join(df.alias("DO_Borough"), cond2, "inner")\
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+--------+------------+------------+----------+-------------+--------------------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|LocationID| Borough|        Zone|service_zone|LocationID|      Borough|                Zone|service_zone|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+----------+--------+------------+------------+----------+-------------+--------------------+------------+
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|        55|Brooklyn|Coney Island|   Boro Zone|         1|          EWR|      Newark Airport|         EWR|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|      

In [177]:
#.withColumn("PU_loc").
cond1 = [df2.PULocationID == df.LocationID]
cond2= [df2.DOLocationID == df.LocationID]
df2\
    .join((df.Borough).alias("PU_Borough"), cond1, "left")\
    .join((df.Borough).alias("DO_Borough"), cond2, "inner")\
    .show()

TypeError: Column is not iterable

In [176]:
cond1 = [df2.PULocationID == df.LocationID]
cond2= [df2.DOLocationID == df.LocationID]
df2\
    .select("*")\
    .join(df.select("Borough").alias("PU_Borough"), cond1, "left")\
    .show()

AnalysisException: Resolved attribute(s) LocationID#1042 missing from pickup_datetime#242,SR_Flag#246,hvfhs_license_num#240,dropoff_datetime#243,PULocationID#244L,Borough#1043,DOLocationID#245L,dispatching_base_num#241 in operator !Join LeftOuter, (PULocationID#244L = cast(LocationID#1042 as bigint)).;;
!Join LeftOuter, (PULocationID#244L = cast(LocationID#1042 as bigint))
:- Project [hvfhs_license_num#240, dispatching_base_num#241, pickup_datetime#242, dropoff_datetime#243, PULocationID#244L, DOLocationID#245L, SR_Flag#246]
:  +- Relation[hvfhs_license_num#240,dispatching_base_num#241,pickup_datetime#242,dropoff_datetime#243,PULocationID#244L,DOLocationID#245L,SR_Flag#246] parquet
+- SubqueryAlias PU_Borough
   +- Project [Borough#1043]
      +- Relation[LocationID#1042,Borough#1043,Zone#1044,service_zone#1045] csv


In [202]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

In [207]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [208]:
year = 2021

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7


processing data for 2021/8
processing data for 2021/9


AnalysisException: Path does not exist: file:/home/greg_dtc1/notebook/data/raw/green/2021/09;

In [212]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [213]:
year = 2021

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7


processing data for 2021/8
processing data for 2021/9


AnalysisException: Path does not exist: file:/home/greg_dtc1/notebook/data/raw/yellow/2021/09;

In [209]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [210]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [214]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [215]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [216]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [217]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [218]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [219]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [220]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [221]:
df_trips_data.registerTempTable('trips_data')

In [222]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [223]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [224]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')

In [229]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')



In [277]:
df2.tail(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02879', pickup_datetime=datetime.datetime(2021, 2, 27, 13, 34, 40), dropoff_datetime=datetime.datetime(2021, 2, 27, 13, 50, 54), PULocationID=137, DOLocationID=249, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02869', pickup_datetime=datetime.datetime(2021, 2, 27, 9, 5, 28), dropoff_datetime=datetime.datetime(2021, 2, 27, 9, 56, 46), PULocationID=21, DOLocationID=174, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 28, 7, 4, 26), dropoff_datetime=datetime.datetime(2021, 2, 28, 7, 16, 11), PULocationID=229, DOLocationID=170, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 27, 22, 30, 5), dropoff_datetime=datetime.datetime(2021, 2, 27, 22, 55, 6), PULocationID=42, DOLocationID=107, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pi

In [284]:
df2.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [281]:
df2\
    .withColumn("Combo", F.concat(F.coalesce(df2.PULocationID, F.lit("Unknown")),
                                  F.lit("/"),
                                  F.coalesce(df2.DOLocationID, F.lit("Unknown"))))\
    .groupBy("Combo").count()\
    .sort(F.desc("count"))\
    .show()

+-------+-----+
|  Combo|count|
+-------+-----+
|  76/76|45041|
|  26/26|37329|
|  39/39|28026|
|  61/61|25976|
|  14/14|17934|
|    7/7|14688|
|129/129|14688|
|  42/42|14481|
|  37/37|14424|
|  89/89|13976|
|216/216|13716|
|  35/35|12829|
|132/265|12542|
| 188/61|11814|
|  95/95|11548|
|  36/37|11491|
|  37/36|11487|
| 61/188|11462|
| 61/225|11342|
|188/188|11308|
+-------+-----+
only showing top 20 rows



In [288]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [295]:
df_zone\
    .filter(df_zone.LocationID=="55")\
    .show()

+----------+--------+------------+------------+
|LocationID| Borough|        Zone|service_zone|
+----------+--------+------------+------------+
|        55|Brooklyn|Coney Island|   Boro Zone|
+----------+--------+------------+------------+



In [278]:
cond1 = [df2.PULocationID == df_zone.LocationID]
cond2= [df2.DOLocationID == df_zone.LocationID]
df2\
    .join(df_zone.withColumnRenamed('Borough', 'PU_Borough'), cond1, "left")\
    .join(df_zone.withColumnRenamed('Borough', 'DO_Borough'), cond2, "left")\
    .select("hvfhs_license_num", "pickup_datetime", "PU_Borough", "DO_Borough")\
    .tail(5)

[Row(hvfhs_license_num='HV0003', pickup_datetime=datetime.datetime(2021, 2, 27, 13, 34, 40), PU_Borough='Manhattan', DO_Borough=None),
 Row(hvfhs_license_num='HV0003', pickup_datetime=datetime.datetime(2021, 2, 27, 9, 5, 28), PU_Borough='Brooklyn', DO_Borough=None),
 Row(hvfhs_license_num='HV0005', pickup_datetime=datetime.datetime(2021, 2, 28, 7, 4, 26), PU_Borough='Manhattan', DO_Borough=None),
 Row(hvfhs_license_num='HV0005', pickup_datetime=datetime.datetime(2021, 2, 27, 22, 30, 5), PU_Borough='Manhattan', DO_Borough=None),
 Row(hvfhs_license_num='HV0003', pickup_datetime=datetime.datetime(2021, 2, 27, 10, 30, 24), PU_Borough='Brooklyn', DO_Borough=None)]

In [ ]:
F.coalesce
+"/"+df_result.DO_Borough)

In [299]:
df2\
    .select("PULocationID", "DOLocationID")\
    .show()

+------------+------------+
|PULocationID|DOLocationID|
+------------+------------+
|         163|         235|
|         225|         181|
|         260|          95|
|         235|          60|
|          55|          55|
|          74|          81|
|         239|         231|
|          95|         196|
|         241|         265|
|          90|         137|
|           7|           7|
|         231|         246|
|         108|          29|
|          11|         112|
|         229|         234|
|          10|         218|
|          49|         232|
|         125|          17|
|          47|          68|
|          41|         223|
+------------+------------+
only showing top 20 rows



In [305]:
df_zone_PU = df_zone\
                .withColumnRenamed('Zone', 'PU_Zone') \
                .withColumnRenamed('LocationID', 'PU_LocationID') 

df_zone_DO= df_zone\
                .withColumnRenamed('Zone', 'DO_Zone') \
                .withColumnRenamed('LocationID', 'DO_LocationID') 
df_result= df2\
    .join(df_zone_PU, df2.PULocationID == df_zone_PU.PU_LocationID, "inner")\
    .join(df_zone_DO, df2.DOLocationID == df_zone_DO.DO_LocationID, "inner")

In [306]:
df_result\
    .select("PULocationID", "DOLocationID", "PU_Zone", "DO_Zone")\
    .show()

+------------+------------+--------------------+--------------------+
|PULocationID|DOLocationID|             PU_Zone|             DO_Zone|
+------------+------------+--------------------+--------------------+
|         163|         235|       Midtown North|University Height...|
|         225|         181|  Stuyvesant Heights|          Park Slope|
|         260|          95|            Woodside|        Forest Hills|
|         235|          60|University Height...|   Crotona Park East|
|          55|          55|        Coney Island|        Coney Island|
|          74|          81|   East Harlem North|         Eastchester|
|         239|         231|Upper West Side S...|TriBeCa/Civic Center|
|          95|         196|        Forest Hills|           Rego Park|
|         241|         265|Van Cortlandt Vil...|                  NA|
|          90|         137|            Flatiron|            Kips Bay|
|           7|           7|             Astoria|             Astoria|
|         231|      

In [307]:
df_result\
    .withColumn("Combo", F.concat(F.coalesce(df_result.PU_Zone, F.lit("Unknown")),
                                  F.lit("/"),
                                  F.coalesce(df_result.DO_Zone, F.lit("Unknown"))))\
    .groupBy("Combo").count()\
    .sort(F.desc("count"))\
    .show()

+--------------------+-----+
|               Combo|count|
+--------------------+-----+
|East New York/Eas...|45041|
|Borough Park/Boro...|37329|
|   Canarsie/Canarsie|28026|
|Crown Heights Nor...|25976|
| Bay Ridge/Bay Ridge|17934|
|Jackson Heights/J...|14688|
|     Astoria/Astoria|14688|
|Central Harlem No...|14481|
|Bushwick South/Bu...|14424|
|Flatbush/Ditmas P...|13976|
|South Ozone Park/...|13716|
|Brownsville/Brown...|12829|
|      JFK Airport/NA|12542|
|Prospect-Lefferts...|11814|
|Forest Hills/Fore...|11548|
|Bushwick North/Bu...|11491|
|Bushwick South/Bu...|11487|
|Crown Heights Nor...|11462|
|Crown Heights Nor...|11342|
|Prospect-Lefferts...|11308|
+--------------------+-----+
only showing top 20 rows



In [ ]:
cond1 = [df2.PULocationID == PU_zone.LocationID]
cond2= [df2.DOLocationID == DO_zone.LocationID]
df_result= df2\
    .join(df_zone.withColumnRenamed('Zone', 'PU_Zone').alias("PU_zone"), cond1, "inner")\
    .join(df_zone.withColumnRenamed('Zone', 'DO_Zone').alias("DO_zone"), cond2, "inner")
#     .select("hvfhs_license_num", "pickup_datetime", "PU_Borough", "DO_Borough")

df_result\
    .withColumn("Combo", F.concat(F.coalesce(df_result.PU_Zone, F.lit("Unknown")),
                                  F.lit("/"),
                                  F.coalesce(df_result.DO_Zone, F.lit("Unknown"))))\
    .groupBy("Combo").count()\
    .sort(F.desc("count"))\
    .show()

In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [271]:
df2\
    .select('hvfhs_license_num')\
    .groupBy('hvfhs_license_num').agg(F.count("*"))\
    .show()

+-----------------+--------+
|hvfhs_license_num|count(1)|
+-----------------+--------+
|           HV0004|   86018|
|           HV0005| 3237166|
|           HV0003| 8290758|
+-----------------+--------+

